In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import re
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("stopwords")
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Faris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Faris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Faris\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Faris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_train = pd.read_csv("updated_train.csv")
df_test = pd.read_csv("updated_test.csv")

In [4]:
df_train.isnull().sum()

ID        0
text      0
target    0
dtype: int64

In [5]:
df_train.head()

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0


In [6]:
df_train["text"] = df_train["text"].astype("str")

In [7]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, "", input_txt)
    
    return input_txt

In [8]:
df_train['text'] = np.vectorize(remove_pattern)(df_train['text'], "@[\w]*")
df_train["text"] = df_train['text'].str.replace("[^a-zA-Z#]", " ")
df_test["text"] = np.vectorize(remove_pattern)(df_test["text"], "@[\w]*")
df_test["text"] = df_test["text"].str.replace("[^a-zA-Z#]", " ")

In [9]:
df_train.head()

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0


In [10]:
tokens = []

for line in df_train.text:
    token = nltk.word_tokenize(line)
    tokens.append(len(token))
    
df_train['No_Token'] = tokens

def split(text):
    return text

char = []
for x in df_train.text.iloc[0:]:
    chara = split(x)
    char.append(len(chara))

df_train["No_Characters"] = char

In [11]:
df_train.head()

,ID,text,target,No_Token,No_Characters
0,train_0,The bitcoin halving is cancelled due to,1,7,39
1,train_1,MercyOfAllah In good times wrapped in its gran...,0,20,117
2,train_2,Days No Digital India No Murder of e learn...,1,19,93
3,train_3,India is likely to run out of the remaining RN...,1,23,112
4,train_4,In these tough times the best way to grow is t...,0,26,121


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_df = 0.9, min_df = 0.1)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
cv = CountVectorizer(min_df = 1, max_df = 2)

In [15]:
X = df_train.drop(["ID", "text", "target"], axis=1)
y = df_train.target

In [16]:
x_test = df_test.drop(["ID"], axis=1)

In [17]:
x_test = cv.fit_transform(df_test["text"])

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)

## Model Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=100, random_state=0, n_estimators=1000)
RF = clf.fit(X_train, y_train)

In [22]:
RF_train = RF.predict(X_train)

print("Akurasi Train = {}".format(
            accuracy_score(RF_train, y_train)*100)
     )
f1_score = metrics.f1_score(RF_train, y_train, average="macro")
print(" Skor Klasifikasi Train F1: {}".format(f1_score*100))

RF_test = RF.predict(X_test)

print("Akurasi Test = {}".format(
            accuracy_score(RF_test, y_test)*100)
     )

f1_score = metrics.f1_score(RF_test, y_test, average="macro")
print(" Skor Klasifikasi Test F1: {}".format(f1_score*100))

Akurasi Train = 75.40789784819107
 Skor Klasifikasi Train F1: 75.35657149875455
Akurasi Test = 55.765595463138
 Skor Klasifikasi Test F1: 55.76543739279588


In [23]:
RF_test1 = RF.predict(X_test).clip(0.1, 0.70)

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [25]:
rfc = RandomForestClassifier(random_state=42)

In [34]:
param_grid1 = {
    "n_estimators": [200,500],
    "max_features": ["auto","sqrt","log2"],
    "max_depth": [4,5,6,7,8],
    "criterion": ["gini","entropy"]
}


In [35]:
CV_rfc = GridSearchCV(estimator = rfc, param_grid=param_grid1, cv=5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'

In [36]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'auto',
 'n_estimators': 200}

In [37]:
rfc1 = RandomForestClassifier(random_state=42, max_features="auto", n_estimators=200,
                             max_depth=7, criterion="gini")

In [38]:
rfc1 = clf.fit(X_train, y_train)

In [39]:
pred = rfc1.predict(X_test).clip(0.1, 0.7)

In [41]:

Rfc_train = rfc1.predict(X_train)

print('Train accuracy = {}'.format(
        accuracy_score(Rfc_train , y_train) * 100)
     )
f1_score = metrics.f1_score(Rfc_train, y_train, average='macro')
print(' F1 Train classification score: {}'.format(f1_score* 100))

Rfc_test= rfc1.predict(X_test)

print('Test accuracy = {}'.format(
        accuracy_score(Rfc_test, y_test) * 100)
     )
f1_score = metrics.f1_score(Rfc_test, y_test, average='macro')
print(' F1 Test classification score: {}'.format(f1_score* 100))

Train accuracy = 75.40789784819107
 F1 Train classification score: 75.35657149875455
Test accuracy = 55.765595463138
 F1 Test classification score: 55.76543739279588


In [42]:
Rfc_test = rfc1.predict(X_test)
print(Rfc_test)

[0 0 0 ... 1 0 0]


In [48]:
log_loss(valid_tag, pred, eps=1e-15)

NameError: name 'valid_tag' is not defined

In [50]:
tokens = []
def num_token(df):
    for line in df:
        token = nltk.word_tokenize(line)
        tokens.append(len(token))
        df['No_Token'] = tokens

def split(text):
    return text

char = []
def char(df):
    for x in df.iloc[0:]:
        chara = split(x)
        char.append(len(chara))
        df['No_Characters'] = char

In [53]:
df = input(str(input))
def data_pro(text):
    df = {"text": [text]}
    df = pd.DataFrame(df)
    return df

tokens = []
def num_token(df):
    for line in df.text:
        token = nltk.word_tokenize(line)
        tokens.append(len(token))
        df["No_Token"] = pd.Series(tokens)
        
character = []
def char(df):
    for x in df.text.iloc[0:]:
        chara = split(x)
        character.append(len(chara))
        df["No_Characters"] = character
        
df = data_pro(text)
num_token(df)
char(df)
df = df.drop(["text"], axis=1)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x000001F6172DFB48>>character


NameError: name 'text' is not defined

In [56]:
RF.predict(df)

ValueError: could not convert string to float: 'character'

In [57]:
import pickle

In [58]:
filename = "model.pkl"
pickle.dump(RF, open(filename, "wb"))

## Gradient classifier

In [65]:
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier


clf = ensemble.GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=120,
                                                  n_iter_no_change=None,
                                                  presort='deprecated',
                                                  random_state=None,
                                                  subsample=1.0, tol=0.0001,
                                                  validation_fraction=0.1,
                                                  verbose=0, warm_start=False)
gbc = clf.fit(train_set, train_tag)

NameError: name 'train_set' is not defined